In [1]:
from main import *
import pandas as pd

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\Users\Martijn\miniconda3\envs\BEP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
image = "vraag 14.jpg"
prompt, response = run_program(image)

In [2]:
truth = pd.read_csv("ground-truth.csv")
results = truth.copy(deep = True)
results["Answer(word)"] = None
results["Answer(letter)"] = None
results["Speed"] = None

In [3]:
truth.head(5)

,Filename,Answer(letter),Answer(word),Speed
0,vraag 1.jpg,A,Brake,40
1,vraag 2.jpg,A,Brake,80
2,vraag 3.jpg,B,Release Accelerator,25
3,vraag 4.jpg,A,Brake,40
4,vraag 5.jpg,C,Nothing,30


In [4]:
# len(truth.index)

for row in range(0, 3):
    tru_row = truth.loc[row]
    res_row = results.loc[row]

    image = tru_row["Filename"]
    prompt, response, car_speed = run_program(image)
    resp_char = response[0]

    if resp_char == 'A':
        resp_word = "Brake"
    elif resp_char == 'B':
        resp_word = "Release Accelerator"
    elif resp_char == 'C':
        resp_word = "Nothing"
    else:
        resp_word = "unknown"

    res_row["Answer(letter)"] = resp_char
    res_row["Speed"] = car_speed
    res_row["Answer(word)"] = resp_word





image 1/1 C:\Users\Martijn\Desktop\coding-projects\BEP\GPT-DT\images\vraag 1.jpg: 384x640 1 outside-view, 1 rear-view, 1 speed, 172.3ms
Speed: 0.0ms preprocess, 172.3ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to tri-crop\predict
YOLOv5  2023-5-10 Python-3.9.16 torch-2.0.0 CPU



'utf-8' codec can't decode byte 0xff in position 0: invalid start byte


Fusing layers... 
Model summary: 476 layers, 76226352 parameters, 0 gradients, 110.1 GFLOPs
Adding AutoShape... 


OSError: No file or directory found at model_remv1.keras